# Avalanche Risk Prediction
### Jaymin West
### Spring, 2023

The orginal idea for this project was to create a model that couuld predict the layers within a snow pack in a given region. Initially, I thought that I would be able to use weather data to achive this goal. However, it became evident that the data I was hoping to use was too sparse and what I thought was a datascience problem was actually a meteorology problem. As such, I had to adjust the scope of this project to focusing more on predicting the overall avalanche risk in a given area. 

### Step 1: Gathering The Data

The area I chose to use was that covered by the Northwest Avalanche Center (NWAC). NWAC covers the western half of Washington and into Oregon. NWAC is a small part of Avalanche.org who host the websites for dozens of avalnche centers around the country. This is important as I built a webscraper to scrape all the archival data from NWAC's site and that webscraper could be expaned to work on all of the other avalanche centers. The code I wrote for the webscraper can be found in the webscraper.py file. It goes through a table of dates and gets the corresponding risk, area, and forecast for each date. The forecast comes in three sections, all of which go into varying levels of detail about the observed conditions and risks in the area

In [1]:
import webscraper

# Getting the avalanche data at Snoqualmie Pass for the current season: 
ws = webscraper.Webscraper("Snoqualmie Pass")
ws.open_archive_page()
reports_data = ws.scrape_daily_reports()
ws.to_csv(reports_data)

KeyboardInterrupt: 

### Step 2: Processing The Data

I was not expecting this to be one of the more time consuming section of the project but it was. It took me a long time trying to figure out how to best use my text data to predict the avalanche risk in the future. I figured I would have to do a lot of preprocessing to get my text into a state that would be useful for the computer. I took all of the text data, removed the stop words (including some custom stop words), lemmatized and lowercased everything, then combined it all into one column. All of these steps can be seen in the code below which is taken from my utils.py file.

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim import corpora
from gensim.utils import simple_preprocess
import pandas as pd

def preprocess_text_column(text):
    """
    Preprocesses a text column for LDA analysis.
    """
    stop_words = set(stopwords.words('english'))
    # adding days of the week to stop words
    stop_words.update(['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'])
    # adding months to stop words
    stop_words.update(['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
                       'september', 'october', 'november', 'december'])
    lemmatizer = WordNetLemmatizer()
    
    try: 
        words = word_tokenize(text.lower())
        words = [w for w in words if not w in stop_words]
        words = [lemmatizer.lemmatize(w) for w in words]
        words = simple_preprocess(str(words), deacc=True)

        return ' '.join(words)
    except:
        return text
    
def clean_raw_webscraper_data(fname):
    """
    Takes in a file of raw avalanche reports data from the webscraper, and returns a dataframe of the cleaned
    """
    # Reading in the CSV file:
    all_data = pd.read_csv(fname)
    # Adding Column Names:
    all_data.columns = ['date', 'zone', 'overall_risk', 'above_treeline_risk', 'near_treeline_risk', 'below_treeline_risk', 'bottom_line_text', 'problem_type_text', 'forecast_discussion_text']
    # Adding a column for the combined text of all 3 text columns:
    all_data['combined_text'] = all_data['bottom_line_text'] + all_data['problem_type_text'] + all_data['forecast_discussion_text']
    # Converting date column to datetime:
    all_data['date'] = pd.to_datetime(all_data['date'])
    
    # Processing all text columns:
    text_coloumns = ['bottom_line_text', 'problem_type_text', 'forecast_discussion_text', 'combined_text']
    
    for column in text_coloumns:
        all_data[column] = all_data[column].apply(preprocess_text_column)

    all_data['zone'] = all_data['zone'].str.lower()
    all_data['overall_risk'] = all_data['overall_risk'].str.lower()

    # Mapping risk ratings to numbers:
    rating_mapping = {
        "extreme": 5.0,
        "high": 4.0,
        "considerable": 3.0,
        "moderate": 2.0,
        "low": 1.0,
        "no rating": 0.0
    }

    all_data['overall_risk'] = all_data['overall_risk'].map(rating_mapping)

    return all_data

### Step 3: Making Predictions and Using LLM's

My intiail plan for this project involved using an LSTM or some similar model to predict the avalanche risk for the next week. While I 

In [2]:
import keys
from langchain import VectorDBQA, OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.document_loaders import DataFrameLoader
import pandas as pd

api_key = keys.OPENAI_KEY

# Loading the data:
all_data = pd.read_csv('output_data/incomplete_All_Zones_Current_Season_reports_data.csv')
all_data.columns = columns=['date', 'zone', 'overall_risk', 'above_treeline_risk', 'near_treeline_risk', 'below_treeline_risk', 'bottom_line_text', 'problem_type_text', 'forecast_discussion_text']
all_data['combined_text'] = all_data['bottom_line_text'] + all_data['problem_type_text'] + all_data['forecast_discussion_text']

# Loading the data:
loader = DataFrameLoader(all_data, 'combined_text')
documents = loader.load()

# Splitting the data:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
docs = text_splitter.split_documents(documents)

# Getting the embeddings:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

# Using QDrant (Vector DB)
qdrant = Qdrant.from_documents(
    docs, embeddings,
    path="qdrant_data",
    collection_name="nwac_qdrant",
)

llm = OpenAI(openai_api_key=api_key)
qa = VectorDBQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    vectorstore=qdrant,
    return_source_documents=False
)

query = "What conditions cause the highest avalanche rating on Steven's Pass?"
results = qa.run(query)
print(results)

/home/jaymin/Desktop/Spring 2023/CPSC325/SnowPackPrediction/.venv/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:201: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


 Overnight strong wind and heavy snow will make it easy to trigger large avalanches near and above treeline. Avoid being on or under slopes over 35 degrees where the wind has drifted recent snow into deeper slabs. If the new snow slides easily in snowpack tests or recent avalanches at any elevation, stick to lower-angle slopes.
